In [ ]:
import requests
r = requests.get("https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json")
data = r.json()

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(data['pokemon'])
df.head()

,id,num,name,img,type,height,weight,candy,candy_count,egg,spawn_chance,avg_spawns,spawn_time,multipliers,weaknesses,next_evolution,prev_evolution
0,1,001,Bulbasaur,http://www.serebii.net/pokemongo/pokemon/001.png,"[Grass, Poison]",0.71 m,6.9 kg,Bulbasaur Candy,25.0,2 km,0.690,69.0,20:00,[1.58],"[Fire, Ice, Flying, Psychic]","[{'num': '002', 'name': 'Ivysaur'}, {'num': '0...",NaN
1,2,002,Ivysaur,http://www.serebii.net/pokemongo/pokemon/002.png,"[Grass, Poison]",0.99 m,13.0 kg,Bulbasaur Candy,100.0,Not in Eggs,0.042,4.2,07:00,"[1.2, 1.6]","[Fire, Ice, Flying, Psychic]","[{'num': '003', 'name': 'Venusaur'}]","[{'num': '001', 'name': 'Bulbasaur'}]"
2,3,003,Venusaur,http://www.serebii.net/pokemongo/pokemon/003.png,"[Grass, Poison]",2.01 m,100.0 kg,Bulbasaur Candy,NaN,Not in Eggs,0.017,1.7,11:30,None,"[Fire, Ice, Flying, Psychic]",NaN,"[{'num': '001', 'name': 'Bulbasaur'}, {'num': ..."
3,4,004,Charmander,http://www.serebii.net/pokemongo/pokemon/004.png,[Fire],0.61 m,8.5 kg,Charmander Candy,25.0,2 km,0.253,25.3,08:45,[1.65],"[Water, Ground, Rock]","[{'num': '005', 'name': 'Charmeleon'}, {'num':...",NaN
4,5,005,Charmeleon,http://www.serebii.net/pokemongo/pokemon/005.png,[Fire],1.09 m,19.0 kg,Charmander Candy,100.0,Not in Eggs,0.012,1.2,19:00,[1.79],"[Water, Ground, Rock]","[{'num': '006', 'name': 'Charizard'}]","[{'num': '004', 'name': 'Charmander'}]"


In [ ]:
df.dtypes

id                  int64
num                object
name               object
img                object
type               object
height             object
weight             object
candy              object
candy_count       float64
egg                object
spawn_chance      float64
avg_spawns        float64
spawn_time         object
multipliers        object
weaknesses         object
next_evolution     object
prev_evolution     object
dtype: object

In [ ]:
#C.head()
type(df['multipliers'][0])
type(df['multipliers'][1])
type(df['multipliers'][2])

NoneType

In [ ]:
#pokemons with spawn rate less thanm 5%
df['name'][df['spawn_chance']<0.05]

1         Ivysaur
2        Venusaur
4      Charmeleon
5       Charizard
7       Wartortle
          ...    
145       Moltres
147     Dragonair
148     Dragonite
149        Mewtwo
150           Mew
Name: name, Length: 65, dtype: object

In [ ]:
#pokemons with less than 4 weaknesses
mask = df.weaknesses.apply(lambda x: len(x)<4)
df[mask]["name"]

3      Charmander
4      Charmeleon
5       Charizard
6        Squirtle
7       Wartortle
          ...    
145       Moltres
146       Dratini
147     Dragonair
149        Mewtwo
150           Mew
Name: name, Length: 102, dtype: object

In [ ]:
#pokemons with no multipliers
NoneType = type(None)
mask = df.multipliers.apply(lambda x: type(x)==NoneType)
df[mask]["name"]

2        Venusaur
5       Charizard
8       Blastoise
11     Butterfree
14       Beedrill
          ...    
144        Zapdos
145       Moltres
148     Dragonite
149        Mewtwo
150           Mew
Name: name, Length: 81, dtype: object

In [ ]:
type(df['prev_evolution'][1]) == list

True

In [ ]:
#this function returns boolean values based on no of evolution
#i.e if a given pokemon has sum of prev_evolution and next_evolution more than 2 : False else True
def map_evol_count(x,y):
  evol_cnt = 1
  if (type(x) != list) & (type(y)!= list):
    return True
  elif type(x) == list:
    if type(y) == list:
      evol_cnt = evol_cnt + len(x) + len(y)
    else:
      evol_cnt += len(x)
  elif type(y)== list:
    evol_cnt += len(y)
  if evol_cnt > 2:
    return False
  else:
    return True


In [ ]:
#pokemon which do not have more than 2 evolution
mask = df.apply(lambda a:map_evol_count(x=a['prev_evolution'],y=a['next_evolution']),axis=1)
df[mask]["name"]

18      Rattata
19     Raticate
20      Spearow
21       Fearow
22        Ekans
         ...   
143    Articuno
144      Zapdos
145     Moltres
149      Mewtwo
150         Mew
Name: name, Length: 102, dtype: object

In [ ]:
df['spawn_time'].replace('N/A','00:00',inplace=True)
df['spawn_time']

0      20:00
1      07:00
2      11:30
3      08:45
4      19:00
       ...  
146    06:41
147    11:57
148    23:38
149    00:00
150    00:00
Name: spawn_time, Length: 151, dtype: object

In [ ]:
df['spawn_time'] = pd.to_timedelta("00:"+df['spawn_time']).astype('timedelta64[s]').astype(int)

In [ ]:
#pokemons with spawn time less than 300 sec
df['name'][df['spawn_time'] < 300]

6       Squirtle
8      Blastoise
10       Metapod
12        Weedle
13        Kakuna
         ...    
143     Articuno
144       Zapdos
145      Moltres
149       Mewtwo
150          Mew
Name: name, Length: 81, dtype: object

In [ ]:
df.columns

Index(['id', 'num', 'name', 'img', 'type', 'height', 'weight', 'candy',
       'candy_count', 'egg', 'spawn_chance', 'avg_spawns', 'spawn_time',
       'multipliers', 'weaknesses', 'next_evolution', 'prev_evolution'],
      dtype='object')

In [ ]:
mask = df.type.apply(lambda x: len(x)>=2)
df[mask]['name']

0       Bulbasaur
1         Ivysaur
2        Venusaur
5       Charizard
11     Butterfree
          ...    
141    Aerodactyl
143      Articuno
144        Zapdos
145       Moltres
148     Dragonite
Name: name, Length: 62, dtype: object